# Imports

In [1]:
# Basic imports
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats

# Magic load kedro
%load_ext kedro.extras.extensions.ipython

2022-06-18 00:55:47,655 - kedro.extras.extensions.ipython - INFO - Updated path to Kedro project: C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kaggle\americanExpress\Kadro
2022-06-18 00:55:49,023 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-18 00:55:49,397 - kedro.extras.extensions.ipython - INFO - ** Kedro project kaggle
2022-06-18 00:55:49,397 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


# Functions

In [ ]:
# Correlation betwen binary and continuous distributions functions
def biserialCorr(data, x, binarY):
  # convert both variables to arrays
  xArray = np.array(data[x], dtype=int).ravel()
  binarYArray = np.array(data[binarY], dtype=int).ravel()
  
  # return biseralCorr
  return stats.pointbiserialr(binarYArray, xArray)

def kruskalDiffTest(data, x, binarY):
  # take the two numpy arrays, one where the variable is one, and one where the variable is 0
  xClass_0 = np.array(data.query(f"{binarY} == 0")[x], dtype=int).ravel()
  xClass_1 = np.array(data.query(f"{binarY} == 1")[x], dtype=int).ravel()
  
  return stats.kruskal(xClass_0, xClass_1)

def bivariateTest(data, x, binarY, showOnly=False):
  # Make biserial correlation and kruskal hypthothesis test
  biserial = biserialCorr(data, x, binarY)
  kruskal = kruskalDiffTest(data, x, binarY)
  
  # if the user only wants toi print the values
  if showOnly:
    print(f"Biserial Correlation Result: {biserial}")
    print(f"Kruskal Diff Result: {kruskal}")
    
    return None
  
  return biserial, kruskal

# Amex fraud prediciton project:

### In this project American Express provided tons of data in order to build a fraud prediciton model, the itention of this notebook attachded to Kedro is to do more free data exploration and test "pre-node" scripts, like data pipelines, models, etc.

# Data loading

loading the train joined data, wich is te size optimezed train, with labels

In [3]:
%reload_kedro

df = catalog.load("train_joined")

2022-06-18 00:55:49,566 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kaggle\americanExpress\Kadro
2022-06-18 00:55:49,925 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-18 00:55:50,172 - kedro.extras.extensions.ipython - INFO - ** Kedro project kaggle
2022-06-18 00:55:50,176 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-06-18 00:55:50,176 - kedro.io.data_catalog - INFO - Loading data from `train_joined` (ParquetDataSet)...


# Data Size

In [ ]:
df.shape

(5531451, 192)

# Data Types

Write the type, names, and example of the columns records to a simple txt file, to facilitate correct type analysis

All data types look right

In [ ]:
# Lets do a relatory to see if everthing is right

relatory = {}

for column in df.columns:
    # Make empty dict for column key
    relatory[column] = {}

    # Assign name
    relatory[column]["name"] = column

    # Assign type
    relatory[column]["type"] = df[column].dtype

    # Assign exampes
    relatory[column]["examples"] = df[column].sample(5)


for col in df.columns:
    with open("report.txt", "a") as writer:
        writer.write(str(relatory[col]))
        writer.write("======" * 100)

# Data Cleaning

The data has null values? No

In [15]:
# Since there is a very big number of columns, lets the couting into a series
nullSeries = df.isnull().sum()

In [21]:
for line in nullSeries:
    if line > 0:
        print(f"Column: {line.index} | Nulls: {line}")

# No null values on trainining data

# Decriptive Analysis

## Categorical Features

---> No cat fetures

## Numerical Features

In [11]:
colGroups = {}

for col in df.columns:
    #Take col first letter
    letter = col[0]

    # check if this letter is incol groups jeys
    if letter in colGroups.keys():
        colGroups[letter] += [col]

    elif letter.upper() == letter:
        colGroups[letter] = [col]

# Print a descr analysis for each letter group with theis titles
for lettergroup in colGroups:
    print("x_x_x" * 30)
    for col in colGroups[lettergroup]:
        print("=-=" * 5)
        print(df[col].describe())


x_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_xx_x_x
=-==-==-==-==-=
count    5.531451e+06
mean     2.157567e+00
std      3.915208e-01
min      1.000000e+00
25%      2.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      4.000000e+00
Name: B_30, dtype: float64
=-==-==-==-==-=
count    5.531451e+06
mean     3.715714e+00
std      1.581884e+00
min      1.000000e+00
25%      3.000000e+00
50%      3.000000e+00
75%      4.000000e+00
max      8.000000e+00
Name: B_38, dtype: float64
=-==-==-==-==-=
count    5.531451e+06
mean     1.240101e-01
std      2.119869e-01
min     -7.588799e+00
25%      8.863645e-03
50%      3.132968e-02
75%      1.259019e-01
max      1.324060e+00
Name: B_1, dtype: float64
=-==-==-==-==-=
count    5.531451e+06
mean     6.210803e-01
std      4.019848e-01
min     -5.000000e-01
25%      1.050166e-01
50%      8.143263e-01
75%      1.002400e+00
max      1.010000e+00
N

# Correlation with target analysis

In [9]:
correlations = {
    "Column": [],
    "BiCorr" : [],
    "Kruskal": []
}

# For every column in dataframe columns
for column in df.columns:

    # verify if the column is not and id or the target
    if column != "target" and "ID" not in column:

        # do the two statistics tests
        biseralCor, kruskal = bivariateTest(data=df, x=column, binarY="target")

        # append the results into relatory
        correlations["Column"].append(column)
        correlations["BiCorr"].append(biseralCor[0])
        correlations["Kruskal"].append(kruskal[0])

# Deleting cross-correlated variables (The variables with the most target correalation estays on the dataframe)